## 背景

在大规模数据训练时，数据可以达到百万级量级。如果计算整个训练集，来获得仅仅一个参数的更新速度就太慢了。一个常用的方法是[Mini-Batch Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)，它计算训练集中的小批量（mini-batche）数据从而实现神经网络参数快速更新。

这节我们会先定义一个softmax分类器，然后使用[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)的训练集来训练这个神经网络。最后使用测试集来验证神经网络的准确率，不同的是我们将通过使用Mini-Batch Gradient Descent来实现网络参数快速更新，这样神经网络的准确率可以达到40%。

## 跟上一节相同，引入依赖并构建神经网络

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC10`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC10`
import $ivy.`org.nd4j:nd4j-native-platform:0.7.2`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.Layer.Tape
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq
import scala.util.Random

pprintConfig() = pprintConfig().copy(height = 2)

import $file.ReadCIFAR10ToNDArray
import $file.Utils

def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

implicit def optimizer: Optimizer = new LearningRate {
  def currentLearningRate() = 0.00001
}

//10 label of CIFAR10 images(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10

def createMyNeuralNetwork(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val initialValueOfWeight = Nd4j.randn(NumberOfPixels, NumberOfClasses) * 0.001
  val weight: INDArray @Symbolic = initialValueOfWeight.toWeight
  val result: INDArray @Symbolic = input dot weight
  softmax.compose(result)
}
val myNeuralNetwork = createMyNeuralNetwork

def lossFunction(implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val expectedOutput = pair.tail.head
  val probabilities = myNeuralNetwork.compose(input)

  -(expectedOutput * log(probabilities)).mean
}

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.Differentiabl

## 运用Mini-Batch Gradient Descent

类似前一节，我们需要训练神经网络，但是跟上一节不同的是，这次我们的训练数据是随机读取的，上一节是反复训练同一批数据集。训练神经网络并观察每次训练`loss`的变化，`loss`的变化趋势是降低，但是不是每次都降低(前途是光明的，道路是曲折的)。

### 根据数组读取和处理数据，然后训练神经网络

In [2]:
val MiniBatchSize = 256
val NumberOfPixels: Int = 3072

def trainData(randomIndexArray: Array[Int]): Double = {
  val trainNDArray :: expectLabel :: shapeless.HNil =
    ReadCIFAR10ToNDArray.getSGDTrainNDArray(randomIndexArray)

  val input =
    trainNDArray.reshape(MiniBatchSize, NumberOfPixels)

  val expectLabelVectorized =
    Utils.makeVectorized(expectLabel, NumberOfClasses)

  lossFunction.train(input :: expectLabelVectorized :: HNil)
}

MiniBatchSize: Int = 256
defined function trainData

### 每个[epoch](http://stackoverflow.com/questions/4752626/epoch-vs-iteration-when-training-neural-networks)打乱一次序列,生成随机数组

In [3]:
val random = new Random

val lossSeq =
  (
    for (iteration <- 0 to 50) yield {
      val randomIndex = random
        .shuffle[Int, IndexedSeq](0 until 10000) //https://issues.scala-lang.org/browse/SI-6948
        .toArray
      for (times <- 0 until 10000 / MiniBatchSize) yield {
        val randomIndexArray =
          randomIndex.slice(times * MiniBatchSize,
                            (times + 1) * MiniBatchSize)
          val loss = trainData(randomIndexArray)
          if(times == 3 & iteration % 5 == 4){
            println("at epoch " + (iteration / 5 + 1) + " loss is :" + loss)
          }
          loss
      }
    }
  ).flatten

val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

at epoch 1 loss is :0.21049327850341798
at epoch 2 loss is :0.20709555149078368
at epoch 3 loss is :0.1957709789276123
at epoch 4 loss is :0.1974302649497986
at epoch 5 loss is :0.19418622255325318
at epoch 6 loss is :0.19290707111358643
at epoch 7 loss is :0.19335532188415527
at epoch 8 loss is :0.18547534942626953
at epoch 9 loss is :0.18905540704727172
at epoch 10 loss is :0.1805393099784851


random: Random = scala.util.Random@62f3ad76
lossSeq: IndexedSeq[Double] = Vector(
  0.229979944229126,
...
plot: Seq[Scatter] = List(
  Scatter(
...
res2_3: String = "plot-1041885403"

## 准备和处理测试集

类似[前一节](https://thoughtworksinc.github.io/DeepLearning.scala/demo/SoftmaxLinearClassifier.html)，我们从CIFAR10 database中读取和处理测试数据的图片和对应的标签信息。但是这次我们在这里只读取测试集即可，训练集已在训练时随机读取。

In [4]:
val testNDArray =
   ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

val testData = testNDArray.head

val testExpectResult = testNDArray.tail.head

val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.0...
vectorizedTestExpectResult: INDArray = [[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
...

## 验证神经网络预测准确率

跟上一节相同，我们使用测试数据来验证神经网络的预测结果并计算准确率。这次准确率应该会有所上升，最终结果在40%左右。

In [5]:
val right = Utils.getAccuracy(myNeuralNetwork.predict(testData), testExpectResult)
println(s"the result is $right %")

the result is 40.0 %


right: Double = 40.0

## 总结

在这节中我们学到了：

* Mini-Batch Gradient Descent
* epoch


[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/MiniBatchGradientDescent.scala)